In [68]:
pip install betfairlightweight pandas numpy flumine streamlit


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [81]:
# import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime as dt
import json
from urllib.parse import quote_plus

# Load credentials
with open('credentials.json') as f:
    cred = json.load(f)

my_username = cred['username']
my_password = cred['password']
my_app_key = cred['app_key']

# Absolute paths to cert files
crt_path = os.path.abspath("certs/client-2048.crt")
key_path = os.path.abspath("certs/client-2048.key")

# Debug: check files exist
print("CRT exists:", os.path.exists(crt_path))
print("KEY exists:", os.path.exists(key_path))



# Initialize API client with certificate login
trading = betfairlightweight.APIClient(
    username=my_username,
    password=my_password,
    app_key=my_app_key,
    cert_files=(crt_path, key_path)
)

# Attempt login
try:
    trading.login()  # This will now succeed if cert/key are correct
    print("Logged in successfully!")
    print("Session token:", trading.session_token)  # Optional debug
except betfairlightweight.exceptions.LoginError as e:
    print("Login failed:", e)
except Exception as e:
    print("Unexpected error:", e)


CRT exists: True
KEY exists: True
Logged in successfully!
Session token: 2Q2+tzvjIS2pKHaMvZjrEpKayvkMthIMirIqrui4ANU=


In [70]:
from betfairlightweight.filters import market_filter

# Filter for UK horse racing
uk_horse_filter = market_filter(
    event_type_ids=['7'],       # Horse Racing
    market_countries=['GB'],    # UK markets
    market_type_codes=['WIN','PLACE']  # Only Win and Place markets
)

events = trading.betting.list_events(filter=uk_horse_filter)
print(f"Found {len(events)} UK Horse Racing events")

# Example: show first 10 events
for event in events[:12]:
    print(event.event.name, event.event.id, event.event.open_date)


Found 10 UK Horse Racing events
Chelmsford City 4th Dec 35021286 2025-12-04 16:30:00
Wetherby 6th Dec 35028118 2025-12-06 11:10:00
Exeter 5th Dec 35024647 2025-12-05 12:13:00
Chepstow 6th Dec 35028119 2025-12-06 11:55:00
Sandown 6th Dec 35028001 2025-12-06 12:10:00
Sedgefield 5th Dec 35024638 2025-12-05 11:48:00
Sandown 5th Dec 35024637 2025-12-05 12:35:00
Newcastle 5th Dec 35024650 2025-12-05 15:15:00
Wolverhampton 6th Dec 35028122 2025-12-06 16:30:00
Aintree 6th Dec 35028121 2025-12-06 11:48:00


In [71]:
from betfairlightweight.filters import market_filter

# Pick an event to explore
event_id = events[0].event.id

# Create a market filter for this event
event_market_filter = market_filter(
    event_ids=[event_id],
    market_type_codes=['WIN','PLACE']  # only filter parameters go here
)

# Get market catalogue with market_projection separately
market_catalogues = trading.betting.list_market_catalogue(
    filter=event_market_filter,
    max_results=20,
    market_projection=['RUNNER_DESCRIPTION','MARKET_START_TIME']
)

print(f"\nMarkets for {events[0].event.name}:")
for market in market_catalogues:
    print(market.market_name, market.market_id)



Markets for Chelmsford City 4th Dec:
To Be Placed 1.251260157
1m Nov Stks 1.251260155
To Be Placed 1.251260163
5f Hcap 1.251260162
To Be Placed 1.251260171
6f Hcap 1.251260169
To Be Placed 1.251260178
6f Hcap 1.251260176
To Be Placed 1.251260184
7f Hcap 1.251260183
To Be Placed 1.251260191
1m2f Hcap 1.251260190
1m2f Hcap 1.251260197
To Be Placed 1.251260198


In [72]:
# Pick the first market
market_id = market_catalogues[0].market_id

# Create a mapping of runner_id -> runner_name from the market catalogue
runner_map = {runner.selection_id: runner.runner_name for runner in market_catalogues[0].runners}

# Get market book (odds, status, etc.)
market_books = trading.betting.list_market_book(
    market_ids=[market_id],
    price_projection={'priceData': ['EX_BEST_OFFERS']}
)

market_book = market_books[0]

print(f"\nRunners in {market_catalogues[0].market_name}:")
for runner in market_book.runners:
    name = runner_map.get(runner.selection_id, "Unknown")
    print(f"{name} - Last Price Traded: {runner.last_price_traded}, Available to Back: {runner.ex.available_to_back}, Available to Lay: {runner.ex.available_to_lay}")



Runners in To Be Placed:
Venetia - Last Price Traded: 1.4, Available to Back: [<betfairlightweight.resources.bettingresources.PriceSize object at 0x000002137FD2F110>, <betfairlightweight.resources.bettingresources.PriceSize object at 0x000002137FD2EC50>, <betfairlightweight.resources.bettingresources.PriceSize object at 0x000002137FD2CF90>], Available to Lay: [<betfairlightweight.resources.bettingresources.PriceSize object at 0x000002137FD2F6D0>, <betfairlightweight.resources.bettingresources.PriceSize object at 0x000002137FD2F890>, <betfairlightweight.resources.bettingresources.PriceSize object at 0x000002137FD2D090>]
Capichera - Last Price Traded: 1.46, Available to Back: [<betfairlightweight.resources.bettingresources.PriceSize object at 0x000002137FD2F750>, <betfairlightweight.resources.bettingresources.PriceSize object at 0x000002137FD2D310>, <betfairlightweight.resources.bettingresources.PriceSize object at 0x000002137FD2E9D0>], Available to Lay: [<betfairlightweight.resources.b

In [ ]:
from datetime import datetime, date, timezone
from betfairlightweight import filters

# --- Specify the date ---
today = date.today()  # Change to any date: date(2025, 12, 3)
start_dt = datetime.combine(today, datetime.min.time()).replace(tzinfo=timezone.utc)
end_dt   = datetime.combine(today, datetime.max.time()).replace(tzinfo=timezone.utc)

#
market_filter = filters.market_filter(
    event_type_ids=['7'],      # Horse Racing
    market_type_codes=['WIN'],
    market_countries=['GB'],
    market_start_time={
        'from': start_dt.isoformat(),
        'to': end_dt.isoformat()
    }
)


markets = trading.betting.list_market_catalogue(
    filter=market_filter,
    max_results=500,  # Increase if needed
    market_projection=['EVENT', 'MARKET_START_TIME']
)


markets_sorted = sorted(markets, key=lambda m: m.market_start_time)


def get_course(market):
    if hasattr(market, "event") and market.event:
        if hasattr(market.event, "venue") and market.event.venue:
            return market.event.venue
        if hasattr(market.event, "name") and market.event.name:
            return market.event.name
    return "Unknown Course"


print(f"\nALL UK HORSE RACING WIN MARKETS for {today}:\n")
for m in markets_sorted:
    start_local = m.market_start_time.astimezone()  # Convert to local time
    time_str = start_local.strftime("%H:%M")
    
    race_name = m.market_name
    course = get_course(m)
    
    print(f"{time_str}  -  {course}  -  {race_name}")



ALL UK HORSE RACING WIN MARKETS for 2025-12-04:

17:30  -  Chelmsford City  -  1m Nov Stks
18:00  -  Chelmsford City  -  5f Hcap
18:30  -  Chelmsford City  -  6f Hcap
19:00  -  Chelmsford City  -  6f Hcap
19:30  -  Chelmsford City  -  7f Hcap
20:00  -  Chelmsford City  -  1m2f Hcap
20:30  -  Chelmsford City  -  1m2f Hcap


In [ ]:
# Fetch markets with runner details
markets_with_runners = trading.betting.list_market_catalogue(
    filter=market_filter,
    max_results=500,
    market_projection=['EVENT', 'MARKET_START_TIME', 'RUNNER_DESCRIPTION', 'RUNNER_METADATA']
)

# Sort markets by start time and pick first 3
markets_sorted_with_runners = sorted(markets_with_runners, key=lambda m: m.market_start_time)
first_3_races = markets_sorted_with_runners[:3]

# Print horses for each race (with cloth number, trainer, and jockey)
print(f"\nUK HORSE RACING — WIN MARKETS FOR {today}\n")

for idx, market in enumerate(first_3_races, 1):
    start_local = market.market_start_time.astimezone()
    time_str = start_local.strftime("%H:%M")
    course = get_course(market)
    race_name = market.market_name

    print(f"\n{'='*60}")
    print(f"Race {idx}: {time_str} - {course} - {race_name}")
    print(f"{'='*60}")

    for runner in market.runners:
        horse_name = runner.runner_name
        metadata = runner.metadata or {}
        cloth_number = metadata.get("CLOTH_NUMBER") or "N/A"
        trainer = metadata.get("TRAINER_NAME") or "N/A"
        jockey = metadata.get("JOCKEY_NAME") or "N/A"

        print(f"  Cloth #{cloth_number}: {horse_name} — Trainer: {trainer} — Jockey: {jockey}")




UK HORSE RACING — WIN MARKETS FOR 2025-12-04


Race 1: 17:30 - Chelmsford City - 1m Nov Stks
  Cloth #10: Venetia — Trainer: R M Beckett — Jockey: Rossa Ryan
  Cloth #4: Capichera — Trainer: E Walker — Jockey: K Shoemark
  Cloth #9: Symbol Of Majesty — Trainer: J & T Gosden — Jockey: R Havlin
  Cloth #5: Cherry Baker — Trainer: James Owen — Jockey: P Cosgrave
  Cloth #7: Molly Omally — Trainer: A M Balding — Jockey: Rob Hornby
  Cloth #8: Siam Ruby — Trainer: R M Beckett — Jockey: H Crouch
  Cloth #3: Blackisthenewblack — Trainer: A M Balding — Jockey: Jason Watson
  Cloth #1: Al Amirah — Trainer: I Mohammed — Jockey: S D Bowen
  Cloth #6: Love Me Bae — Trainer: Alice Haynes — Jockey: K T O'Neill
  Cloth #2: Alice De Clare — Trainer: J R Fanshawe — Jockey: Non Runner

Race 2: 18:00 - Chelmsford City - 5f Hcap
  Cloth #8: Cherry Cobbler — Trainer: S C Williams — Jockey: Rossa Ryan
  Cloth #4: Twilight Fun — Trainer: C Banham — Jockey: Joey Haynes
  Cloth #5: Almaty Star — Trainer: R M 

In [80]:
from urllib.parse import quote_plus
from IPython.display import display, Markdown

horses = [
    "Nottodaybobo",
    "Seahouses"

]

# Generate clickable Timeform search links
for horse in horses:
    # URL-encode the horse name
    search_url = f"https://www.timeform.com/horse-racing/search?q={quote_plus(horse)}"
    
    # Display as clickable Markdown link
    display(Markdown(f"- [{horse} Timeform Search]({search_url})"))



- [Nottodaybobo Timeform Search](https://www.timeform.com/horse-racing/search?q=Nottodaybobo)

- [Seahouses Timeform Search](https://www.timeform.com/horse-racing/search?q=Seahouses)